In [2]:
%pip install faiss-cpu 
%pip install langchain_ollama 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader('speech.txt')
documents=loader.load()
text_splitter = CharacterTextSplitter(chunk_size=25,chunk_overlap=10)
docs = text_splitter.split_documents(documents)

In [10]:
## Load the model and db
modelName = 'llama3.2:1b'
embeddings = OllamaEmbeddings(model=modelName)
db=FAISS.from_documents(docs,embeddings)
db

In [11]:
## querying
query='''FAISS'in temel özellikleri nelerdir?'''
docs = db.similarity_search(query, k=3)
docs

[Document(id='b39b4771-ec4d-47c6-b31d-cd9b9d60d6bc', metadata={'source': 'speech.txt'}, page_content="FAISS'in temel özellikleri şunlardır:"),
 Document(id='b94474dc-93cc-4914-a4c2-5845fd984489', metadata={'source': 'speech.txt'}, page_content='FAISS, büyük veri kümelerinde benzer öğeleri hızlıca bulmak için güçlü bir araçtır.'),
 Document(id='6e97706a-0106-4e64-ad06-408084115469', metadata={'source': 'speech.txt'}, page_content='FAISS (Facebook AI Similarity Search), büyük ölçekli vektör arama yapabilen bir kütüphanedir. Yüksek boyutlu vektörleri hızlı bir şekilde aramak ve benzerlik ölçümü yapmak için optimize edilmiştir.')]

In [12]:
docs[0].page_content

"FAISS'in temel özellikleri şunlardır:"

In [15]:
## retriever
query='''FAISS'in temel özellikleri nelerdir?'''
retriever = db.as_retriever()
retriever.invoke(query, k=3)
docs[0].page_content

"FAISS'in temel özellikleri şunlardır:"

In [18]:
### simialrity search with score
query='''FAISS'in temel özellikleri nelerdir?'''
docs_with_score = db.similarity_search_with_score(query, k=3, return_score=True)
docs_with_score

[(Document(id='b39b4771-ec4d-47c6-b31d-cd9b9d60d6bc', metadata={'source': 'speech.txt'}, page_content="FAISS'in temel özellikleri şunlardır:"),
  0.3131186),
 (Document(id='b94474dc-93cc-4914-a4c2-5845fd984489', metadata={'source': 'speech.txt'}, page_content='FAISS, büyük veri kümelerinde benzer öğeleri hızlıca bulmak için güçlü bir araçtır.'),
  0.4854807),
 (Document(id='6e97706a-0106-4e64-ad06-408084115469', metadata={'source': 'speech.txt'}, page_content='FAISS (Facebook AI Similarity Search), büyük ölçekli vektör arama yapabilen bir kütüphanedir. Yüksek boyutlu vektörleri hızlı bir şekilde aramak ve benzerlik ölçümü yapmak için optimize edilmiştir.'),
  0.49819574)]

In [24]:
## search by vectors
embeddings_vector=embeddings.embed_query(query)
db.similarity_search_by_vector(embeddings_vector, k=3)
#db.similarity_search_with_score_by_vector(embeddings_vector, k=3)

[Document(id='b39b4771-ec4d-47c6-b31d-cd9b9d60d6bc', metadata={'source': 'speech.txt'}, page_content="FAISS'in temel özellikleri şunlardır:"),
 Document(id='b94474dc-93cc-4914-a4c2-5845fd984489', metadata={'source': 'speech.txt'}, page_content='FAISS, büyük veri kümelerinde benzer öğeleri hızlıca bulmak için güçlü bir araçtır.'),
 Document(id='6e97706a-0106-4e64-ad06-408084115469', metadata={'source': 'speech.txt'}, page_content='FAISS (Facebook AI Similarity Search), büyük ölçekli vektör arama yapabilen bir kütüphanedir. Yüksek boyutlu vektörleri hızlı bir şekilde aramak ve benzerlik ölçümü yapmak için optimize edilmiştir.')]

In [ ]:
## save db
db.save_local('faiss_db')

In [ ]:
## load db
new_db = FAISS.load_local('faiss_db',embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(query, k=3)
docs

[Document(id='b39b4771-ec4d-47c6-b31d-cd9b9d60d6bc', metadata={'source': 'speech.txt'}, page_content="FAISS'in temel özellikleri şunlardır:"),
 Document(id='b94474dc-93cc-4914-a4c2-5845fd984489', metadata={'source': 'speech.txt'}, page_content='FAISS, büyük veri kümelerinde benzer öğeleri hızlıca bulmak için güçlü bir araçtır.'),
 Document(id='6e97706a-0106-4e64-ad06-408084115469', metadata={'source': 'speech.txt'}, page_content='FAISS (Facebook AI Similarity Search), büyük ölçekli vektör arama yapabilen bir kütüphanedir. Yüksek boyutlu vektörleri hızlı bir şekilde aramak ve benzerlik ölçümü yapmak için optimize edilmiştir.')]

: 